In [39]:
import requests
import re
import json
import csv
import subprocess
import constants

In [ ]:
# How "forward thinking" is the locality (can be measured by internet use, etc)?
# How many other people in your locality go to school?
# How does your probable household structure impact academic performance?
# Does family size matter?
# Does unemployment in your neighborhood affect your mentality towards education?
# Does high commuting time to work matter since it probably affects family time?
# Does parent occupation matter?
# Family income and costs (mortgage, rent, expenses as a percent fo income, etc)
# Percent below poverty line families
# Does diversity in your neighborhood matter?

In [ ]:
# B09018: RELATIONSHIP TO HOUSEHOLDER FOR CHILDREN UNDER 18 YEARS IN HOUSEHOLDS
# B08303:  TRAVEL TIME TO WORK
# B11016: HOUSEHOLD TYPE BY HOUSEHOLD SIZE
# B09005: HOUSEHOLD TYPE FOR CHILDREN UNDER 18 YEARS IN HOUSEHOLDS (EXCLUDING HOUSEHOLDERS, SPOUSES, AND UNMARRIED PARTNERS)

In [ ]:
def get_group_variables(group_name_desc: dict):
    general_link = 'https://api.census.gov/data/2018/acs/acs5/groups.json'

    groups = requests.get(general_link).json()['groups']

    group_vars = {}
    for d in groups:
        if d['name'] in group_name_desc:
            # verify that group_name and group_desc match
            assert d['description'].lower() == group_name_desc[d['name']].lower(), f'{d["name"]} and {group_name_desc[d["name"]]} do not match'

            group_link = f'https://api.census.gov/data/2018/acs/acs5/groups/{d["name"]}.json'

            fields = requests.get(group_link).json()['variables']
            estimate_fields = []
            for f in fields:
                if f.endswith('E'): # if field is an "estimate"
                    estimate_fields.append({f: fields[f]['label']})

            group_vars[d['name']] = estimate_fields

In [59]:
def get_column_names(columns):
    column_name_url = 'https://api.census.gov/data/2018/acs/acs5/profile/variables.json'
    d = requests.get(column_name_url, headers=headers).json()['variables']
    return [re.sub('!!', ' | ', d[c]['label']) for c in columns]

def get_raw_data(columns):
    base_url = 'https://api.census.gov/data/2018/acs/acs5'
    data = None
    for c in constants.counties_fips:
        parameters = {
            'for': 'block group:*',
            'in': f'state:{constants.state_fips} county:{c}',
            'get': ','.join(columns),
            'key': constants.key
        }
        if not data:
            data = requests.get(base_url, params=parameters).json()
        else:
            data = requests.get(base_url, params=parameters)[1:]
        print('here')
    return data

In [ ]:
def write_to_csv(data, filename):
    with open(filename, 'w') as f:
        w = csv.writer(f)
        for l in data:
            w.writerow(l)

def generate_create_table_stmt(filename, table_name):
    return subprocess.check_output(
        f'csvsql {csv_file} -i postgresql --tables {table_name}',
        shell=True
    ).decode()

In [ ]:
cols = []
for c in group_vars['B08303']:
    cols.append(c[0])

get_raw_data(constants.state_fips, cols)

In [63]:
base_url = 'https://api.census.gov/data/2018/acs/acs5'
data = None
c = constants.counties_fips[0]
parameters = {
    'for': 'block group:*',
    'in': f'state:{constants.state_fips} county:{c}',
    'get': ','.join(cols),
    'key': constants.key
}
data = requests.get(base_url, params=parameters).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)